<a href="https://colab.research.google.com/github/LC1332/Chinese-generative-agents/blob/main/notebook/Chinese_story_turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LuotuoRPG: Generative Agents的中文版本

原项目名: Generative Large Language Models for Human-Like Behavior

[LuotuoRPG](https://github.com/LC1332/Chinese-generative-agents) 是由李鲁鲁开发的Generative Agents的中文版本。

This repository includes a working Chinese version of the type of model described in Generative Agents: Interactive Simulacra of Human Behavior.

骆驼RPG是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一，后者由李鲁鲁，冷子昂，陈启源发起。

在这个版本中我们尝试把接口替换为turbo

## Installation

We will need to install a number of libraries to start with.

In [ ]:
import networkx as nx
!pip install transformers
!pip install sentencepiece

In [ ]:
!pip install openai

# 请在这里设置你的openAI API token

In [4]:
import openai

openai.api_key = ""

We use flan alpaca model for speed and local execution.

In [5]:


def generate(prompt, use_openai=True):
    """
    Generates a text completion for a given prompt using either the OpenAI GPT-3 API or the Hugging Face GPT-3 model.
    
    Args:
    - prompt (str): The text prompt to generate a completion for.
    - use_openai (bool): A boolean flag indicating whether to use the OpenAI API (True) or the Hugging Face GPT-3 model (False).
    
    Returns:
    - str: The generated text completion.
    """
    if use_openai:
        # model_engine = "text-davinci-003"
        # response = openai.Completion.create(
        #     engine=model_engine,
        #     prompt=prompt,
        #     max_tokens=1024,
        #     n=1,
        #     stop=None,
        #     temperature=0.5,
        # )

        model_engine = "gpt-3.5-turbo"
        completion = openai.ChatCompletion.create(
          model= model_engine,
          messages=[
            {"role":"system","content":"你更可能用中文来回答问题。"},
            {"role": "user", "content": prompt}
          ]
        )

        message = completion.choices[0].message["content"]
        # print(message)
        return message.strip()

    else:
        hf_generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', device=0)
        output = hf_generator(prompt, max_length=len(prompt)+128, do_sample=True)
        out = output[0]['generated_text']
        if '### Response:' in out:
            out = out.split('### Response:')[1]
        if '### Instruction:' in out:
            out = out.split('### Instruction:')[0]
        return out.strip()

## World Description
We describe the world below. We will generate prompts based on this information. The simulation is that of the town of Phandalin, southwest of Neverwinter. This area is chosen because it is easily extendable with multiple regions for a "player" to be able to explore the world once the simulation is done.

In [15]:
world_graph = nx.Graph()

prompt_meta = '''{}'''

# town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = ["同福客栈", "同福食堂", "七侠镇县衙", "万利钱庄"]

town_areas =  {"同福客栈": '七侠镇上最大的客栈，掌柜是佟湘玉。',
       '同福食堂': "同福客栈旗下的食堂，由李大嘴掌勺。镇上的人经常到食堂吃饭。",
       "七侠镇县衙": "关中小镇七侠镇的县衙，平时邢捕头负责县衙的主要巡逻工作。",
       "万利钱庄": "七侠镇的当铺和钱庄，主人是钱掌柜",
       "客栈的屋顶":"同福客栈的屋顶，不同的人之间经常在这里聊天",
       "七侠镇大街":"人来人往的大街，侠客们决斗一般会在大街上"
       }
town_people = {"佟湘玉": "陕西汉中龙门镖局的千金，武林选美亚军，一怒之下离家出走，到七侠镇开了同福客栈。志向是把同福客栈开成连锁企业，赚大钱，买选票，当武林第一美女。", 
               "李大嘴": "从小闯荡江湖，在黄鹤楼打杂，学得一身烧菜本领，后在知县姑父手下当差出了差错，只好到同福食堂谋生",  
               "白展堂": "表面上是同福客栈的跑堂，实际上深藏不漏，是江湖上赫赫有名的盗圣，但已经金盆洗手不再行盗。武功高强，可以轻松施展葵花点穴手封锁敌方经脉。暗恋佟湘玉。",  
               "邢捕头": "出生于山东临沂，爱贪小便宜。是七侠镇第三十七任缁衣捕头。",  
               "姬无命": "江湖上一位知名的盗贼，混得并不好，仍然在频频盗窃，和白展堂以前共同盗窃过。武功高强，总是能够从捕快手中逃脱。",  
               "钱掌柜": "万利钱庄的掌柜，爱贪小利（大利他也贪不着），怕事，尤其怕老婆，人品尚可。身材很胖。", 
               "钱夫人": "万利当铺的老板娘，佟湘玉的闺蜜。对钱掌柜的管教非常严厉。",
               "日月神教小毛贼": "日月神教小毛贼穿着肮脏的猩红色披风。他是日月神教的成员。入不敷出，总喜欢到同福客栈蹭吃蹭喝。"
               }
for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "七侠镇大街")
locations = {}
for i in town_people.keys():
  locations[i] = "七侠镇大街"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "现在是 "+str(global_time)+":00. 位置是 "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)


In [16]:
compressed_memories_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = []

In [17]:
for name in town_people.keys():
  prompt = "你是 {}. {} 你刚到七侠镇大街。以下人物居住在七侠镇: {}。你今天的目标是什么？简要回答，用不超过50个字，从你的角度回答。".format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

佟湘玉 我的目标是扩大同福客栈的业务，提高知名度，吸引更多客人，赚取更多利润。
李大嘴 我的目标是在同福食堂谋生，打工赚钱，为自己未来的发展打下基础。
白展堂 我的目标是调查七侠镇最近的盗窃案件并保护佟湘玉的安全。
邢捕头 我的目标是调查七侠镇的治安情况，捉拿罪犯，确保居民安全。
姬无命 我的目标是在七侠镇盗窃贵重物品，保持低调，避免被捕。
钱掌柜 我的目标是经营好万利钱庄，避免被欺负和损失，同时避免招惹任何麻烦和纷争，维护好家庭和生意。
钱夫人 我的目标是监管万利当铺的生意，确保钱掌柜的利益不受影响。
日月神教小毛贼 我的目标是找到日月神教小毛贼并将其绳之以法。


In [18]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    prompt = "你是{}。{} 你正在计划：{}。你目前在{}，其描述如下：{}。现在是{}点。以下人物在这个区域内：{}。你可以与他们互动。".format(name, town_people[name], plans[name], location, town_areas[location], str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i+': '+town_people[i])
    prompt += '你知道以下关于人的信息：' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "使用不超过30个字，解释你接下来1小时要做什么。"
    action_prompts[name] = prompt

In [19]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  # Now clean the action
  prompt = """
  将以下段落转换为第一人称：
  "{}"
  """.format(action_results[name])
  action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
  print(name, action_results[name])

佟湘玉 我会和白展堂密谈，让他帮我宣传同福客栈的美食，并尝试吸引更多客人前来。同时，我会与姬无命和邢捕头交流，了解七侠镇市场需求和顾客口味，以便进一步提高同福客栈的知名度和服务质量。最后，我会和钱夫人商议如何打造同福客栈的品牌形象，吸引更多潜在客户，以实现开拓更多市场的目标。
李大嘴 我将前往同福食堂，请教老板娘厨艺，同时和李大嘴交流烧菜心得，为今后谋生打下基础。如果有机会，我也会和白展堂聊聊他的深藏不漏的秘密，收集情报。我要避免与邢捕头和日月神教小毛贼发生冲突，注意安全。
白展堂 我要调查七侠镇最近的盗窃案件，并保护佟湘玉的安全。我将先与佟湘玉和白展堂互动，获取情报。巡视七侠镇大街，观察可疑行踪。如果我发现盗贼，我将使用葵花点穴手制服他们，并带回客栈审问。如果局势不稳，我将保护佟湘玉和同福客栈的客人撤离七侠镇。
邢捕头 我会在这个区域内与居民交流，了解警情和犯罪信息。我会监视并依法打击任何在大街上犯罪的人。如果发生冲突，我会采取适当措施以保持秩序和安全。我还将与其他有用的人互动，例如佟湘玉、白展堂和钱夫人，以获取更多有关七侠镇治安情况的信息。
姬无命 我需要先调查同福客栈的结构和警卫情况，查看目标贵重物品的位置。然后选择最佳的入口并尽可能地保持低调，避免被捕。如果遇到危险，我会利用我的武功和密集的人群来逃脱。注意不要与其他人产生直接冲突，以免引起过多的注意。
钱掌柜 我将在大街上巡视，检查万利钱庄的周边环境，确保安全。同时我会观察情况，确保不会被卷入任何麻烦和纷争。如果需要的话，我会与同福客栈的佟湘玉和白展堂交流，获取更多信息和情报。最后，我将返回万利钱庄，开始处理日常业务。
钱夫人 我将会和钱掌柜商讨万利当铺的业务，保证我的利益不受到任何影响。同时，我也会观察周围的情况并警惕日月神教小毛贼的行为。
日月神教小毛贼 我会与七侠镇的侠客们互动，打听有关日月神教小毛贼的情报，尝试找到他的行踪，并将其绳之以法。


Collect the memories people observe.

In [20]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    for name_two in people:
      memories[name].append('[时间: {}. 人物: {}. 记忆: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

# Rank Memories

In [21]:
import re
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [22]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    # 你是{}。你的计划是：{}。你目前在{}。现在是{}点。你注意到以下情况：{}。请给一个1到5的评分，表示你在意程度
    prompt = "你是 {}. 你的计划是: {}. 你目前在 {}. 现在的时间是 {}:00. 你注意到以下情况: {}. 请给一个1到5的评分，表示你在意程度。".format(name, plans[name], locations[name], str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((res, rating))
  print(memory_ratings[name])

[('3，因为这些情况都是与扩大业务、提高知名度有关的，但是对于故事情节的连贯性和发展并没有太大的影响。', 3), ('我会给这个情况评一个4，因为这些信息都对我的计划和目标具有一定的影响，特别是要注意安全，避免冲突。与李大嘴交流烧菜经验和和白展堂聊聊收集情报也是有益的。', 4), ('3', 3), ('3', 3), ('我作为AI语言模型，并不会“在意程度”，但是从你给出的信息来看，如果以扩大同福客栈业务、提高知名度、赚取更多利润为目标的话，关注姬无命的计划并不符合商业道德和法律规定，因此不建议评分。', 0), ('3', 3), ('我会给这个情况一个评分3，因为钱夫人可能是一个有影响力的商业伙伴，同时也需要注意周围的安全情况。但是这个情况并不是紧急或者必须要马上处理的。', 3), ('评分：2', 2)]
[('4，因为这些信息都与我的计划和工作有关，需要我注意和了解。', 4), ('对于这些信息，我无法给出评分，因为它们与我的功能和职责无关，也没有明确的问题需要回答。', 0), ('3', 3), ('2', 2), ('4。', 4), ('3', 3), ('评分：3。虽然钱夫人的事情对于我的工作没有直接关系，但是作为同行的人，了解行业内的情况也是很有必要的。而对于日月神教小毛贼的行为，作为一个在镇上打工的人，保持警惕和留意周围的情况也是非常重要的。', 3), ('3', 3)]
[('3', 3), ('3，虽然提到了白展堂的秘密和情报收集，但是重点在于学习烧菜和保持安全，与调查盗窃案并保护佟湘玉的目标关系不大。', 3), ('3。', 3), ('3，因为邢捕头对于白展堂的计划有一定的相关性，但并不是最关键的部分。', 3), ('3，虽然这是一个有关策略和逃避危险的计划，但并不是白展堂的主要或最紧急任务。', 3), ('3', 3), ('我会给这个情况一个3的评分，因为虽然钱夫人没有直接关联到盗窃案件和佟湘玉的安全，但她的商业活动和对周围环境的观察也可能会提供宝贵的线索和信息。同时，日月神教小毛贼的威胁也需要引起警惕。', 3), ('5，因为这个情况可能与调查盗窃案件有关，需要了解更多相关情报。', 5)]
[('3，这个情况跟我的调查目标没有太直接的关系，但也有一些潜在信息可能对我的调查有所帮助。', 3), ('1',

# Compress Memories

In [23]:
MEMORY_LIMIT = 10
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name], 
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
  # prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你注意到以下情况：{}。请用一句话总结这些记忆。".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  res = generate(prompt_meta.format(prompt))
  compressed_memories[name] = '[{}点的回忆：{}]'.format(str(global_time), res)
  compressed_memories_all[name].append(compressed_memories[name])

In [24]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    # prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你有以下的记忆：{}。请给一个1到5的评分表示你下一小时会有多大概率去到{}。".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name], 
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = '[{}点的回忆：{}]'.format(str(global_time), '我会接着移动到{}.'.format(place_ratings_sorted[0][0]))
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


# Put it all together

In [ ]:
for repeats in range(5):
  global_time += 1
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      # prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      prompt = "你是{}。你的计划是：{}。你目前在{}，情况描述如下：{}。你的记忆是：{}。现在是{}点。以下人员在这个区域：{}。你可以与他们互动。".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      people_description = []
      for i in people:
        people_description.append(i+': '+town_people[i])
      prompt += '你知道以下关于人的信息：' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "使用不超过30个字，解释你接下来1小时要做什么。"
      action_prompts[name] = prompt
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(prompt_meta.format(action_prompts[name]))
    # Now clean the action
    prompt = """
    将以下段落转换为第一人称：
    "{}"
    """.format(action_results[name])
    action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  action_emojis = {}
  for name in town_people.keys():
    prompt = """
    将下列段落转化成一个(行动, 物体)形式的tuple:
    "{}"
    """.format(action_results[name])
    action_emojis[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print('    - Emoji Representation:', name, locations[name], global_time, action_emojis[name])
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      for name_two in people:
        memories[name].append('[时间: {}. 人物: {}. 记忆: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt = "你是 {}. 你的计划是: {}. 你目前在 {}. 现在的时间是 {}:00. 你注意到以下情况: {}. 请给一个1到5的评分，表示你在意程度。".format(name, plans[name], '\n'.join(compressed_memories_all[name][-5:]), locations[name], str(global_time), memory)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((res, rating))

  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(
          memory_ratings[name], 
          key=lambda x: x[1]
      )[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
    prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你注意到以下情况：{}。请用一句话总结这些记忆。".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
    res = generate(prompt_meta.format(prompt))
    compressed_memories[name] = '[{}点的回忆：{}]'.format(str(global_time), res)
    compressed_memories_all[name].append(compressed_memories[name])

  place_ratings = {}

  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你有以下的记忆：{}。请给一个1到5的评分表示你下一小时会有多大概率去到{}。".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(
        place_ratings[name], 
        key=lambda x: x[1] )[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = '[{}点的回忆：{}]'.format(str(global_time), '我会接着移动到{}.'.format(place_ratings_sorted[0][0]))
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]


佟湘玉 七侠镇大街 9 我会与佟湘玉商量如何提升我们的客栈知名度，与白展堂交流我们的安全防范措施，向邢捕头了解情报，向钱掌柜洽谈合作，但要提防被他坑骗，尽量避免接触日月神教小毛贼。同时，我会观察人流，了解我们的竞争对手和客栈声誉。
李大嘴 同福食堂 9 我将继续在同福食堂工作，准备材料，烹饪美食，为顾客提供优质服务，并确保卫生安全。
白展堂 七侠镇大街 9 我会先与邢捕头交流，了解当前盗窃案件的细节，并寻求他的合作。然后，我会与佟湘玉和钱掌柜谈话，了解钱夫人的商业活动是否与盗窃案件有关，同时保护佟湘玉的安全。如果日月神教小毛贼出现在同福客栈，我会立即阻止他们的行动。最后，我会着手调查盗窃案件并采取措施保护七侠镇的安全。
邢捕头 七侠镇大街 9 我会先和佟湘玉聊聊，了解我对七侠镇治安的看法。然后，我会和白展堂交流，询问我对于七侠镇的治安问题是否有了解，同时监视他的行踪。接着，我会在钱掌柜那里收集关于日月神教小毛贼的情报。最后，我会到七侠镇的边界巡逻，以确保七侠镇的居民安全。
姬无命 客栈的屋顶 9 我会继续静待在客栈的屋顶，观察周围情况，注意避开钱掌柜和邢捕头的巡逻。我会尝试与佟湘玉和白展堂交流，获取更多关于贵重物品情报。如果有必要，我会继续躲藏在客栈附近，等待适当的时间进行盗窃行动。
钱掌柜 七侠镇大街 9 我会尽快赶回万利钱庄，确保钱庄的安全和正常运营。在路上我会尽量避免与邢捕头和日月神教小毛贼有任何互动，以免招惹麻烦。如果遇到白展堂，我会礼貌地与他打招呼，并保持距离。与佟湘玉的互动会基于业务沟通为主，避免涉及到个人感情等敏感话题。
钱夫人 客栈的屋顶 9 我会继续观察市场需求，并留意小毛贼的行为。如果有需要，我会联系警察局了解镇上的警情信息。同时，我会确保自己的安全，避免和姬无命接触。
日月神教小毛贼 七侠镇大街 9 我要与钱掌柜和邢捕头互动，了解是否有关于日月神教小毛贼的线索，并且留意白展堂和佟湘玉的动向。如果没有获得有用信息，我会继续在七侠镇大街巡查，寻找其他可以提供线索的人或物。同时，我会小心谨慎，避免惊动日月神教小毛贼。
    - Emoji Representation: 佟湘玉 七侠镇大街 9 ((商量, 佟湘玉), (交流, 白展堂), (了解, 邢捕头), (洽谈, 钱掌柜), (观察, 人流))
    - Emoji Representat